In [ ]:
When choosing a model two approaches were given to us

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Charger les données
import pandas as pd
df = pd.read_csv("train_cleaned_numeric_labels.csv")

# Transformer les textes en vecteurs TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X_tfidf = vectorizer.fit_transform(df["Text"])

# Séparer les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df["Label"], test_size=0.2, random_state=42)

# Entraîner le modèle Naive Bayes
model_nb = MultinomialNB()
model_nb.fit(X_train, y_train)

# Prédictions
y_pred = model_nb.predict(X_test)

# Évaluer l'accuracy
print(f"Accuracy du modèle Naive Bayes : {accuracy_score(y_test, y_pred):.4f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.model_selection import train_test_split

# Charger les données
df = pd.read_csv("train_cleaned_numeric_labels.csv")

# Tokenisation des textes
tokenizer = Tokenizer(num_words=50000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["Text"])
X_seq = tokenizer.texts_to_sequences(df["Text"])

# Padding des séquences
X_pad = pad_sequences(X_seq, maxlen=200, padding='post', truncating='post')

# Séparer les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X_pad, df["Label"], test_size=0.2, random_state=42)

# Création du modèle LSTM
model_lstm = Sequential([
    Embedding(input_dim=50000, output_dim=128, input_length=200),
    SpatialDropout1D(0.2),
    LSTM(128, return_sequences=False),
    Dense(128, activation="relu"),
    Dense(len(df["Label"].unique()), activation="softmax")
])

# Compilation du modèle
model_lstm.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Entraînement
model_lstm.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Évaluation
loss, acc = model_lstm.evaluate(X_test, y_test)
print(f"Accuracy du modèle LSTM : {acc:.4f}")
